# 데이터 가져오기

### 한국 관광데이터 숙소정보 리스트

In [3]:
mykey = ''

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json

url = f'http://apis.data.go.kr/B551011/KorService1/searchStay1?areaCode=&sigunguCode=&ServiceKey={mykey}&listYN=Y&MobileOS=ETC&MobileApp=AppTest&arrange=A&numOfRows=3900&pageNo=1'
rt = requests.get(url)
items = BeautifulSoup(rt.text).select('item')

/home/encore/miniconda3/envs/lanchain/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [5]:
len(items)

3887

### infoDF 생성 (숙소명- contentid - contenttypeid)

In [11]:
data = []
for x in items:
    title = x.find('title').text
    contentid = x.find('contentid').text
    contenttypeid = x.find('contenttypeid').text
    
    data.append({'title': title, 'contentid': contentid, 'contenttypeid' : contenttypeid})

infoDF = pd.DataFrame(data)

In [12]:
# 파일 저장 
infoDF.to_csv('./data/infoDF.csv', index=False, encoding='utf-8')

In [14]:
infoDF.head(5)

,title,contentid,contenttypeid
0,가경재,2465071,32
1,가락관광호텔,142785,32
2,가락청,2671267,32
3,가람나무,2627867,32
4,가람초연재,1865597,32


## contentid, contenttypeid 를 사용해 각 상세정보 가져오기

### 기본정보 조회 함수

In [23]:
def selectInfo2(contentid, contenttypeid):
    
    all_url = f'''
    https://api.visitkorea.or.kr/openapi/validate/gwrest/KorService1/detailCommon1?ServiceKey=TourAPI&contentTypeId={contenttypeid}&contentId={contentid}&MobileOS=ETC&MobileApp=AppTest&defaultYN=Y&firstImageYN=Y&areacodeYN=Y&catcodeYN=Y&addrinfoYN=Y&mapinfoYN=Y&overviewYN=Y
    '''.strip()
    r = requests.get(all_url)
    items = BeautifulSoup(r.text).select_one('item')
    
    addr1 = items.find('addr1').text
    overview = items.find('overview').text
    imglilk = []
    firstimage = items.find('firstimage').text
    if firstimage != '':
        imglilk.append(firstimage)
    firstimage2 = items.find('firstimage2').text
    
    if firstimage2 != '':
        imglilk.append(firstimage2)
    
    return  addr1, overview, imglilk

### 숙소정보 조회 함수

In [34]:
def sukso_info(contentid, contenttypeid):
    
    all_url = f'''
    https://api.visitkorea.or.kr/openapi/validate/gwrest/KorService1/detailIntro1?ServiceKey=TourAPI&contentTypeId={contenttypeid}&contentId={contentid}&MobileOS=ETC&MobileApp=AppTest
    '''.strip()
    r = requests.get(all_url)
    items = BeautifulSoup(r.text).select_one('item')  
    if not items:
            return '해당 정보가 없습니다.'
        
    # 태그와 컬럼 매핑
    mapping = {
        'roomtype': '객실유형',
        'roomcount': '객실 수',
        'refundregulation': '환불 규정',
        'checkintime': '체크인',
        'checkouttime': '체크아웃',
        'chkcooking': '조리가능여부',
        'parkinglodging': '주차가능여부',
        'pickup': '픽업서비스',
        'accomcountlodging': '전체 수용인원',
        'infocenterlodging': '문의 안내',
        'reservationlodging': '예약 안내',
        'reservationurl': '예약안내 사이트',
        'scalelodging': '총 규모'
    }

    # 정보 추출
    txt = '-숙소 기본정보-\n'
    for tag, col in mapping.items():
        value = items.find(tag).text.strip() if items.find(tag) else ''
        if value:  # 정보가 있으면 추가
            txt += f'{col}: {value}\n'

    return  txt

### 객실정보 조회 함수

In [25]:
def room_info(contentid, contenttypeid):
    room_url = f'''
https://api.visitkorea.or.kr/openapi/validate/gwrest/KorService1/detailInfo1?ServiceKey=TourAPI&contentTypeId={contenttypeid}&contentId={contentid}&MobileOS=ETC&MobileApp=AppTest
    '''.strip()
    r = requests.get(room_url)
    items = BeautifulSoup(r.text).select('item')  
    mapping = {
        'roomtitle': '객실명',
        'roomsize1': '객실크기',
        'roomcount': '객실수',
        'roombasecount': '기준인원',
        'roomintro': '객실소개',
        'roomoffseasonminfee1': '비수기주중최소',
        'roomoffseasonminfee2': '비수기주말최소',
        'roompeakseasonminfee1' : '주중성수기',
        'roompeakseasonminfee2' : '주말성수기',
        'roombathfacility': '목욕시설',
        'roombath': '욕조',
        'roomaircondition': '에어컨',
        'roomtv': 'TV',
        'roompc': 'PC',
        'roominternet': '인터넷',
        'roomrefrigerator': '냉장고',
        'roomhairdryer': '드라이기',
        'roomtable': '테이블',
        'roomtoiletries': '세면도구',
        'roomsofa': '소파'
    }
    
    all_room_info = []
    for idk, item in enumerate(items):
        # 정보 추출
        txt = f'\n**- 객실정보 {idk+1} -**'
        for tag, col in mapping.items():
            value = item.find(tag).text.strip() if item.find(tag) else ''
            if value:  # 정보가 있으면 추가
                txt += f'{col}: {value}\n'
        all_room_info.append(txt)

    return all_room_info


### 데이터를 담을 객체생성

In [35]:
infoData = []

### 위 함수를 사용하여 데이터 추출

In [50]:
from tqdm import tqdm
for x in tqdm(range(3513, 3887)):
    # print(infoDF.loc[x]['title'])
    title = infoDF.loc[x]['title']
    contentid = infoDF.loc[x]['contentid']
    contenttypeid = infoDF.loc[x]['contenttypeid']
    addr1, overview, imglilk = selectInfo2(contentid, contenttypeid)
    txtinfo = sukso_info(contentid, contenttypeid)
    all_room_info = room_info(contentid, contenttypeid)
    infoData.append({'title': title, 'addr1': addr1, 'overview' : overview, 'imglilk' : imglilk, 'txtinfo' : txtinfo, 'all_room_info' : all_room_info})

  0%|          | 0/374 [00:00<?, ?it/s]/home/encore/miniconda3/envs/lanchain/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
100%|██████████| 374/374 [07:01<00:00,  1.13s/it]


In [49]:
x

3514

### 데이터 확인 

In [ ]:
infoData[0]

{'title': '가경재',
 'addr1': '경상북도 안동시 하회남촌길 69-5',
 'overview': '가경재는 안동 하회마을 남쪽에 자리한 초가 한옥이다. 객실은 총 5개로, 안채는 중앙 툇마루를, 사랑채는 중앙 마루를 함께 사용할 수 있다. 군불황토방은 제일 인기가 좋은 객실로, 최소 2일 전에 예약해야 아궁이에 불을 지펴준다. 객실 곳곳 창호지 바른 창 너머 시원한 바람을 맞으며 자연의 경치를 감상할 수 있다. 한옥이지만 에어컨을 설치해 이용객의 편의성을 높였고, 여름철에는 에어컨 이용료가 부과된다.',
 'imglilk': ['http://tong.visitkorea.or.kr/cms/resource/00/2626200_image2_1.jpg',
  'http://tong.visitkorea.or.kr/cms/resource/00/2626200_image3_1.jpg'],
 'txtinfo': '-숙소 기본정보-\n객실유형: 한실\n객실 수: 2\n환불 규정: 7일전 100%, 3일전 50%, 2일전~당일 환불 불가\n체크인: 16:00\n체크아웃: 10:00\n조리가능여부: 불가\n주차가능여부: 가능\n픽업서비스: 가능\n전체 수용인원: 7\n문의 안내: 054-855-8552\n예약 안내: 054-855-8552, 010-3824-2500\n예약안내 사이트: http://ggj.kr/bbs/content.php?co_id=info\n총 규모: 지상 1층\n',
 'all_room_info': ['\n**- 객실정보 1 -**객실명: 사랑채 아랫방\n객실크기: 0\n객실수: 0\n기준인원: 2\n비수기주중최소: 80000\n비수기주말최소: 80000\n주중성수기: 80000\n주말성수기: 80000\n에어컨: Y\nTV: Y\n인터넷: Y\n냉장고: Y\n드라이기: Y\n테이블: Y\n세면도구: Y\n',
  '\n**- 객실정보 2 -**객실명: 군불 황토 아랫방\n객실크기: 0\n객실수: 0\n기준인원: 3\n비수기주중최소: 100000

In [ ]:
len(infoData)


3887

## DF 생성 및 전처리

In [ ]:
suk_data = []
for x in infoData:
    name = x['title']
    address = x['addr1']
    overview = x['overview']
    imglink = x['imglilk']
    generalInfo = x['txtinfo']
    roomInfo = x['all_room_info']
    
    
    suk_data.append({'name': name, 'address': address, 'overview' : overview, 'imglink' : imglink, 'generalInfo' : generalInfo, 'roomInfo' : roomInfo})

suksoDF = pd.DataFrame(suk_data)

## 태그 생성
- 가중치 부여및 메타데이터 생성

In [ ]:
# 태그 추출 함수 (최적화)
def extract_tag(row):
    # 주소 추출
    road_address = row['address']
    
    # 주소가 유효하고 2개 이상의 단어로 구성된 경우 사용
    if pd.notna(road_address) and len(road_address.split()) >= 2:
        return ' '.join(road_address.split()[:2])

    
    # 둘 다 유효하지 않을 경우
    return '없음'

suksoDF['tag'] = suksoDF.apply(lambda row: extract_tag(row), axis=1)

In [ ]:
suksoDF['tag'].unique()

array(['경상북도 안동시', '서울특별시 송파구', '전북특별자치도 전주시', '경기도 파주시', '제주특별자치도 서귀포시',
       '강원특별자치도 인제군', '강원특별자치도 정선군', '경기도 구리시', '경기도 평택시', '경상남도 하동군',
       '경기도 양평군', '경상북도 성주군', '충청북도 청주시', '강원특별자치도 평창군', '경상남도 남해군',
       '경기도 가평군', '경상남도 고성군', '강원특별자치도 영월군', '강원특별자치도 홍천군', '경상남도 통영시',
       '서울특별시 강남구', '서울특별시 서초구', '강원특별자치도 강릉시', '경상북도 문경시', '전북특별자치도 순창군',
       '강원특별자치도 춘천시', '전라남도 보성군', '인천광역시 강화군', '전라남도 해남군', '경상남도 창원시',
       '강원특별자치도 화천군', '경상북도 포항시', '서울특별시 영등포구', '강원특별자치도 원주시', '충청남도 태안군',
       '충청남도 보령시', '경상남도 거제시', '경상남도 거창군', '서울특별시 마포구', '광주광역시 동구',
       '대구광역시 중구', '서울특별시 용산구', '전북특별자치도 군산시', '전북특별자치도 부안군', '충청남도 홍성군',
       '울산광역시 동구', '인천광역시 연수구', '경상북도 경주시', '경기도 포천시', '충청남도 공주시',
       '경기도 용인시', '강원특별자치도 철원군', '경기도 부천시', '전라남도 장흥군', '경기도 고양시',
       '서울특별시 종로구', '전라남도 고흥군', '전북특별자치도 고창군', '인천광역시 부평구', '전라남도 담양군',
       '강원특별자치도 삼척시', '전라남도 곡성군', '전라남도 구례군', '인천광역시 남동구', '경기도 광명시',
       '경상남도 진주시', '서울특별시 강서구', '인천광역시 중구', '전라남도 여수시', '강원특별자치도 고성군',
    

### 영어주소 -> 한글로 변경처리 

In [ ]:
suksoDF.loc[suksoDF['address'].str.contains('18, Hoegi-ro', na=False), 'address'] = '서울특별시 동대문구 회기로 29길 18'
suksoDF.loc[suksoDF['tag'].str.contains('18, Hoegi-ro', na=False), 'tag'] = '서울특별시 동대문구'

### 태그 강화를 위해 tag내 지역명 추가

In [ ]:
suksoDF.loc[suksoDF['tag'].str.contains('강원도', na=False), 'tag'] = suksoDF['tag'] + ' 강원특별자치도'
suksoDF.loc[suksoDF['tag'].str.contains('강원특별자치도', na=False), 'tag'] = suksoDF['tag'] + ' 강원도'
suksoDF.loc[suksoDF['tag'].str.contains('경기도', na=False), 'tag'] = suksoDF['tag'] + ' 경기'
suksoDF.loc[suksoDF['tag'].str.contains('경상남도', na=False), 'tag'] = suksoDF['tag'] + ' 경상도'
suksoDF.loc[suksoDF['tag'].str.contains('경상북도', na=False), 'tag'] = suksoDF['tag'] + ' 경상도'
suksoDF.loc[suksoDF['tag'].str.contains('광주광역시', na=False), 'tag'] = suksoDF['tag'] + ' 광주'
suksoDF.loc[suksoDF['tag'].str.contains('대구광역시', na=False), 'tag'] = suksoDF['tag'] + ' 대구'
suksoDF.loc[suksoDF['tag'].str.contains('대전광역시', na=False), 'tag'] = suksoDF['tag'] + ' 대전'
suksoDF.loc[suksoDF['tag'].str.contains('부산광역시', na=False), 'tag'] = suksoDF['tag'] + ' 부산'
suksoDF.loc[suksoDF['tag'].str.contains('서울특별시', na=False), 'tag'] = suksoDF['tag'] + ' 서울'
suksoDF.loc[suksoDF['tag'].str.contains('세종특별자치시', na=False), 'tag'] = suksoDF['tag'] + ' 세종'
suksoDF.loc[suksoDF['tag'].str.contains('울산광역시', na=False), 'tag'] = suksoDF['tag'] + ' 울산'
suksoDF.loc[suksoDF['tag'].str.contains('인천광역시', na=False), 'tag'] = suksoDF['tag'] + ' 인천'
suksoDF.loc[suksoDF['tag'].str.contains('전라남도', na=False), 'tag'] = suksoDF['tag'] + ' 전라도'
suksoDF.loc[suksoDF['tag'].str.contains('전북특별자치도', na=False), 'tag'] = suksoDF['tag'] + ' 전라북도 전라도'
suksoDF.loc[suksoDF['tag'].str.contains('제주특별자치도', na=False), 'tag'] = suksoDF['tag'] + ' 제주도 제주'
suksoDF.loc[suksoDF['tag'].str.contains('충청남도', na=False), 'tag'] = suksoDF['tag'] + ' 충청도'
suksoDF.loc[suksoDF['tag'].str.contains('충청북도', na=False), 'tag'] = suksoDF['tag'] + ' 충청도'

In [ ]:
suksoDF['tag']

0               경상북도 안동시 경상도
1               서울특별시 송파구 서울
2       전북특별자치도 전주시 전라북도 전라도
3                 경기도 파주시 경기
4               경상북도 안동시 경상도
                ...         
3882             인천광역시 중구 인천
3883    전북특별자치도 전주시 전라북도 전라도
3884            서울특별시 마포구 서울
3885         강원특별자치도 춘천시 강원도
3886            경상북도 구미시 경상도
Name: tag, Length: 3887, dtype: object

### 태그강화를 위해 소개문구내 특정 문구 추출/입력력

- 소개문에 '한옥' 단어가 있는경우 태그에 추가

In [ ]:
suksoDF.loc[suksoDF['overview'].str.contains('한옥', na=False), 'tag'] = suksoDF['tag'] + ' 한옥'

- 기타 포인트 문구 소개문에서 추출하여 태그 추가하면 좋을것 같음...(시간부족이슈슈)

## 데이터 저장

In [ ]:
suksoDF.to_csv('./data/suksoDF.csv', index=False, encoding='utf-8')

## 데이터 불러오기

In [51]:
suksoDF = pd.read_csv('./data/suksoDF.csv',encoding='utf-8')

# 임베딩 테스트

In [53]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# OpenAI 임베딩 설정
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [54]:
from langchain.schema import Document
# 숙소 기본 정보 DB
db_info = Chroma(
    persist_directory='./sukso_info',
    collection_name='sukso_info',
    embedding_function=embeddings
)
# 기본정보 doc
doc_info = [
    Document(page_content=f"숙소명: {row['name']} \n 주소: {row['address']}  \n 소개문: {row['overview']}  \n 이미지링크: {row['imglink']} ", metadata={"tag": row['tag'] , "idx" : idx})
    for idx, row in suksoDF.iterrows()
]

# 객실 정보용 DB
db_room = Chroma(
    persist_directory='./room_info',
    collection_name='room_info',
    embedding_function=embeddings
)
# 객실정보 doc
room_info = [
    Document(page_content=f"숙소명: {row['name']} \n 숙소정보: {row['generalInfo']}  \n 객실정보: {row['roomInfo']} ", metadata={"tag": row['tag'] , "idx" : idx})
    for idx, row in suksoDF.iterrows()
]


In [55]:
import time
# 생성도큐먼트 db추가 - openai 토큰제한이슈로 배치처리 적용 
batch_size = 50

# 기본정보
for i in tqdm(range(0, len(doc_info), batch_size)):
    batch = doc_info[i:i+batch_size]
    _ = db_info.add_documents(batch) # 출력방지지
    time.sleep(2)

# 객실정보보
for i in tqdm(range(0, len(room_info), batch_size)):
    batch = room_info[i:i+batch_size]
    _ = db_room.add_documents(batch) # 출력방지지
    time.sleep(2)

 65%|██████▌   | 51/78 [03:33<01:42,  3.81s/it]Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: HTTP code 520 from API (<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.openai.com | 520: Web server is returning an unknown error</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-erro

## 프롬프트 테스트

In [56]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
chat = ChatOpenAI(
    model = 'gpt-3.5-turbo'
)

### 정확한 프롬프트 작성을 위한 llm도움 받기

In [62]:
# 사용자 요청 분석 함수
def check_query(user_query):
    prompt = f"""
    다음 문장을 기반으로 두 가지를 확인해 주세요:
    1. 숙소/호텔/민박/호캉스 관련 정보 요청 여부
    2. 구체적인 지역 정보 포함 여부

    **문장: '{user_query}'**

    조건에 따라 아래와 같이 답변하세요:
    - 숙소 정보와 지역 정보가 모두 포함되어 있으면: '0'
    - 둘 중 하나라도 없다면: '없음'
    """
    response = chat.predict(prompt)
    return response.strip()  

In [60]:
# 봇에게 보내는 요청 함수

def question(user_query):
    # 입력받은 문장의 내용을 llm에게 검토요청 
    query_check = check_query(user_query)
    print("분석 결과:", query_check)
    
    #  ChatGPT에 전달할 메시지 구성
    if '없음' in query_check: 
        messages = [
            SystemMessage(content="""
            당신은 숙소/호텔/민박 정보를 제공하는 도우미 챗봇입니다.
            사용자가 원하는 숙소의 종류와 위치를 명확히 말하도록 유도하세요.
            """),
            HumanMessage(content=f"사용자 요청: {user_query}")
        ]
    else :
        try:
            results = db_info.similarity_search(user_query, k=5)
            if not results:
                # 검색 결과가 없을 경우
                return "해당 요청에 맞는 숙소 정보를 찾을 수 없습니다. 다른 키워드로 다시 시도해 주세요."

            # 검색된 결과 통합
            context = "\n".join([result.page_content for result in results])
            print("검색 결과:", context)
            
            messages = [
                SystemMessage(content="""
                당신은 숙소/호텔/민박 정보를 제공하는 도우미 챗봇입니다.
                사용자 요청에 맞춰 검색된 정보를 활용하여 정확한 답변을 제공하세요.
                """),
                HumanMessage(content=f"사용자 질문: {user_query}\n\n🔍 참고 정보:\n{context}\n\n위 정보를 기반으로 답변해 주세요.")
            ]
        except Exception as e:
            return f"검색 중 오류가 발생했습니다: {e}"
        
    #  ChatGPT로부터 답변 받기
    response = chat(messages)

    return response

In [63]:
question("서울에서 호캉스 즐길만한 호텔 추천해줘!")

분석 결과: 1. 숙소/호텔/민박/호캉스 관련 정보 요청 여부: O
2. 구체적인 지역 정보 포함 여부: O

답변: 0
검색 결과: 숙소명: 캄스테이 서울 
 주소: 서울특별시 송파구 석촌호수로12길 3-19 (잠실동)  
 소개문: 캄스테이 서울은 서울 잠실 새내역 근처에 위치한 따스한 채광이 가득한 감성숙소로, 지하철 2, 8, 9호선 모두 이용이 가능해 교통이 편리하다. 복층구조로 1층에는 주방과 거실이 있고, 2층엔 침실과 다락방이 있다. 거실 8인용 넉넉한 테이블에 앉아 캡슐커피를 마시면 그 맛이 일품이다. 석촌호수, 롯데월드, 롯데월드타워, 잠실한강공원 모두 도보로 이용 가능하고, 잠실새내먹자골목과 새마을시장도 인근에 있어 외식 이용도 편리하다.  
 이미지링크: [] 
숙소명: 핸드픽트호텔 서울 
 주소: 서울특별시 동작구 상도로 120 (상도동)  
 소개문: 핸드픽트호텔은 서울지하철 7호선 신대방삼거리역과 장승배기역 사이에 위치한 3성급 호텔이다.
이 호텔은 외관을 벽돌로 마감하고 여러 미술품을 전시하여 고급스러운 느낌을 더하고 있다. 객실은 스튜디오 베이직(트윈, 더블, 킹), 스튜디오 프리미어(트윈, 패밀리), 주니어 스위트(트윈, 킹), 이그젝큐티브 스위트(킹 A~C)로 총 10종류가 있어 인원과 취향에 맞춰 선택할 수 있다. 그리고 한식 레스토랑, 카페 및 캐주얼 다이닝, 플라워샵, 편집샵 등 다양한 편의시설이 있다. 조식은 볼룸에서 유료로 제공된다. 또한 결혼식, 돌잔치, 세미나 등을 개최할 수 있는 연회장도 있다. 매년 가을 서울불꽃축제 개최 시에는 일부 객실에서 프라이빗하게 불꽃축제를 감상할 수 있다.
주변에는 보라매공원, 대방공원 등이 있어 투숙 시 함께 둘러볼 수 있다.  
 이미지링크: [] 
숙소명: 세화호스텔 
 주소: 서울특별시 종로구 삼일대로32가길 50  
 소개문: 서울 종로 중심부에 자리한 세화호스텔은 1~3인 여행객을 위한 온돌룸, 침실룸 등 다양한 객실을 운영한다. 공용 PC, 안마의자, 전자레인지, 세탁기,

AIMessage(content='서울에서 호캉스를 즐길 수 있는 호텔로는 다음을 추천드립니다:\n\n1. **캄스테이 서울**\n   - 주소: 서울특별시 송파구 석촌호수로12길 3-19 (잠실동)\n   - 소개: 채광이 가득한 감성숙소로, 교통이 편리하며 주변 관광지와 음식점이 가까이 있습니다.\n   \n2. **핸드픽트호텔 서울**\n   - 주소: 서울특별시 동작구 상도로 120 (상도동)\n   - 소개: 벽돌로 마감된 외관과 다양한 객실 옵션, 레스토랑, 카페 등의 편의시설을 갖추고 있습니다.\n   \n3. **아만티호텔서울**\n   - 주소: 서울특별시 마포구 월드컵북로 31 아만티호텔 서울\n   - 소개: 홍대입구역 근처에 위치한 4성급 호텔로, 조용하고 쾌적한 분위기를 느낄 수 있으며 다양한 서비스와 부대시설을 제공합니다.\n\n이 세 곳은 호캉스를 즐기기에 적합한 호텔들이니 참고하시기 바랍니다.', additional_kwargs={}, example=False)